# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [17]:
# импортируем нужные библиотеки
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from nltk.stem import SnowballStemmer
snowball_stemmer = SnowballStemmer('russian').stem  # создаем объект SnowballStemmer
from pymystem3 import Mystem
mystem = Mystem().lemmatize # создаем объект Mystem

In [143]:
# прочитаем предоставленный датасет
data = pd.read_csv('../data/data.csv')

# посмотрим информацию о содержании колонок
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [144]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [171]:
pd.DataFrame(data=[
    data.memory_usage(),
    data.astype({'children': 'int8', 'dob_years': 'int8', 'education_id': 'int8', 'family_status_id': 'int8', 'debt': 'int8', 'days_employed': 'float32', 'total_income': 'float32'}).memory_usage(),
    data.memory_usage() - data.astype({'children': 'int8', 'dob_years': 'int8', 'education_id': 'int8', 'family_status_id': 'int8', 'debt': 'int8', 'days_employed': 'float32', 'total_income': 'float32'}).memory_usage()
    ]
).rename({0: 'before_optimization', 1: 'after_optimization', 2: 'compressed_bytes'})

,Index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
before_optimization,128,172200,172200,172200,172200,172200,172200,172200,172200,172200,172200,172200,172200
after_optimization,128,21525,86100,21525,172200,21525,172200,21525,172200,172200,21525,86100,172200
compressed_bytes,0,150675,86100,150675,0,150675,0,150675,0,0,150675,86100,0


**Вывод**

## Шаг 2. Предобработка данных

### Обработка пропусков

**Вывод**

### Замена типа данных

**Вывод**

### Обработка дубликатов

**Вывод**

### Лемматизация

**Вывод**

### Категоризация данных

**Вывод**

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

## Шаг 4. Общий вывод

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.